# Configure credentials

In [ ]:
# api key and aios guid
AIOS_CREDENTIALS = {
  "instance_guid": "AIOS GUID HERE",
  "apikey": "CLOUD API KEY HERE", 
  "url": "https://api.aiopenscale.cloud.ibm.com"
}

In [ ]:
# wml
wml_credentials = {
  
}

In [ ]:
# postgres
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "name": "name here",
    "uri": "uri here"
}

In [ ]:
#cos
COS_CREDENTIALS = {
    'apikey': 'here',
    'iam_service_id': 'here',
    'resource_instance_id': 'here',
    'endpoint': 'https://s3-api.us-geo.objectstorage.softlayer.net',
    'ibm_auth_endpoint': 'https://iam.bluemix.net/oidc/token'
}

COS_BUCKET_NAME = "something-unique-to-you"

# Package installation

In [ ]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install psycopg2-binary | tail -n 1

Restart the kernel to assure the new libraries are being used.

# Load and explore data

## Load the training data from github

In [ ]:
!rm credit_risk_training.csv
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/credit_risk_training.csv

In [ ]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="credit_risk_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

## Store the training data in COS

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
import io, urllib

cos = ibm_boto3.resource('s3',
                      ibm_api_key_id=COS_CREDENTIALS['apikey'],
                      ibm_service_instance_id=COS_CREDENTIALS['resource_instance_id'],
                      ibm_auth_endpoint=COS_CREDENTIALS['ibm_auth_endpoint'],
                      config=Config(signature_version='oauth'),
                      endpoint_url=COS_CREDENTIALS['endpoint'])

buckets = []
for bucket in cos.buckets.all():
        buckets.append(bucket.name)
        
if COS_BUCKET_NAME not in buckets:
    cos.create_bucket(Bucket=COS_BUCKET_NAME)

cos.Bucket(COS_BUCKET_NAME).upload_file('credit_risk_training.csv', 'credit_risk_training.csv')

## Put the training data into a Postgres database

In [ ]:
SCHEMA_NAME = 'credit_risk_data_mart_for_wml'

In [ ]:
from ibm_ai_openscale.utils import create_postgres_schema
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)
TABLE_NAME = "CREDIT_RISK_TRAINING"

In [ ]:
import psycopg2

hostname = POSTGRES_CREDENTIALS['uri'].split('@')[1].split(':')[0]
port = POSTGRES_CREDENTIALS['uri'].split('@')[1].split(':')[1].split('/')[0]
user = POSTGRES_CREDENTIALS['uri'].split('@')[0].split('//')[1].split(':')[0]
password = POSTGRES_CREDENTIALS['uri'].split('@')[0].split('//')[1].split(':')[1]
dbname = 'compose'
conn_string = "host=" + hostname + " port=" + port + " dbname=" + dbname + " user=" + user + " password=" + password

conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("""
DROP TABLE IF EXISTS {}.{}
""".format(SCHEMA_NAME, TABLE_NAME))
cursor.execute("""
CREATE TABLE {}.{}(
    CheckingStatus text,
    LoanDuration integer,
    CreditHistory text,
    LoanPurpose text,
    LoanAmount integer,
    ExistingSavings text,
    EmploymentDuration text,
    InstallmentPercent integer,
    Sex text,
    OthersOnLoan text,
    CurrentResidenceDuration integer,
    OwnsProperty text,
    Age integer,
    InstallmentPlans text,
    Housing text,
    ExistingCreditsCount integer,
    Job text,
    Dependents integer,
    Telephone text,
    ForeignWorker text,
    Risk integer
)
""".format(SCHEMA_NAME, TABLE_NAME))
cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
with open('credit_risk_training.csv', 'r') as f:
    next(f)
    cursor.copy_from(file=f, table='{}.CREDIT_RISK_TRAINING'.format(SCHEMA_NAME), sep=',')
cursor.close()
conn.close()

Print the first row of the training data

In [ ]:
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("""
SELECT * FROM {}.CREDIT_RISK_TRAINING LIMIT 1
""".format(SCHEMA_NAME))
result = cursor.fetchall()
cursor.close()
conn.close()

print(result)

## Explore data

In [ ]:
df_data.printSchema()

In [ ]:
print("Number of records: " + str(df_data.count()))

# Create a model

In [ ]:
# spark_df = sqlCtx.createDataFrame(df_data)
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

model_name = "AIOS Spark German Risk Model - Final"
deployment_name = "AIOS Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

si_CheckingStatus = StringIndexer(inputCol = 'CheckingStatus', outputCol = 'CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol = 'CreditHistory', outputCol = 'CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol = 'LoanPurpose', outputCol = 'LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol = 'ExistingSavings', outputCol = 'ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol = 'EmploymentDuration', outputCol = 'EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol = 'OthersOnLoan', outputCol = 'OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol = 'OwnsProperty', outputCol = 'OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol = 'InstallmentPlans', outputCol = 'InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol = 'Housing', outputCol = 'Housing_IX')
si_Job = StringIndexer(inputCol = 'Job', outputCol = 'Job_IX')
si_Telephone = StringIndexer(inputCol = 'Telephone', outputCol = 'Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol = 'ForeignWorker', outputCol = 'ForeignWorker_IX')

In [ ]:
va_features = VectorAssembler(inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX", "EmploymentDuration_IX", "Sex_IX", \
                                         "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX", "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", \
                                         "LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount", \
                                         "Dependents"], outputCol="features")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(labelCol="Risk", featuresCol="features")

pipeline = Pipeline(stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker, si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,\
                               si_OwnsProperty, si_Sex, si_Telephone, va_features, classifier])
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = MulticlassClassificationEvaluator(labelCol="Risk", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorDT.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save and deploy the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# this is the postgres definition, replace with the cos definition
training_data_reference = {
    "name": "credit risk training reference",
    "connection": POSTGRES_CREDENTIALS,
    "source": {
        "schemaname": SCHEMA_NAME,
        "tablename": TABLE_NAME,
        "type": "postgres"
    }
}

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(model_name),
    wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.8
        }
    ]
}

In [ ]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if model_name == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

In [ ]:
model_uid

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if deployment_name == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=deployment_name, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# Configure OpenScale

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

## Create schema and datamart

In [ ]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

In [ ]:
# if your schema already exists, identify it here
SCHEMA_NAME = 'data_mart_credit_risk'

### Skip this next step if your schema already exists

In [ ]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

### Set up datamart

In [ ]:
ai_client.data_mart.delete()
ai_client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

## Bind machine learning engines

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(wml_credentials))
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

In [ ]:
ai_client.data_mart.bindings.list_assets()

## Subscriptions

### Remove existing credit risk subscriptions

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == model_name:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', model_name)

In [ ]:
# subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
#             source_uid=model_uid,
#             label_column='Risk',
#             prediction_column='prediction',
#             probability_column='probability'
#         ))

subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(model_uid))

if subscription is None:
    # subscription already exists; get the existing one
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == model_name:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

### Score the model

In [ ]:
# wml_client = client.data_mart.bindings.get_native_engine_client(binding_uid=subscription.binding_uid)
credit_risk_scoring_endpoint = None
deployment_uid = subscription.get_deployment_uids()[0]

print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['scoring_url']
        
print(credit_risk_scoring_endpoint)

In [ ]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print(scoring_response)

## Quality and feedback monitoring

### Enable quality monitoring

In [ ]:
subscription.quality_monitoring.enable(problem_type=ProblemType.MULTICLASS_CLASSIFICATION, threshold=0.7, min_records=5)

### Feedback logging

In [ ]:
subscription.feedback_logging.store(
    [
        ["no_checking",28,"outstanding_credit","appliances",5990,"500_to_1000","greater_7",5,"male","co-applicant",3,"car_other",55,"none","free",2,"skilled",2,"yes","yes",1],
["greater_200",22,"all_credits_paid_back","car_used",3376,"less_100","less_1",3,"female","none",2,"car_other",32,"none","own",1,"skilled",1,"none","yes",0],
["no_checking",39,"credits_paid_to_date","vacation",6434,"unknown","greater_7",5,"male","none",4,"car_other",39,"none","own",2,"skilled",2,"yes","yes",1],
["0_to_200",20,"credits_paid_to_date","furniture",2442,"less_100","unemployed",3,"female","none",1,"real_estate",42,"none","own",1,"skilled",1,"none","yes",0],
["greater_200",4,"all_credits_paid_back","education",4206,"less_100","unemployed",1,"female","none",3,"savings_insurance",27,"none","own",1,"management_self-employed",1,"none","yes",0],
["greater_200",23,"credits_paid_to_date","car_used",2963,"greater_1000","greater_7",4,"male","none",4,"car_other",46,"none","own",2,"skilled",1,"none","yes",1],
["no_checking",31,"prior_payments_delayed","vacation",2673,"500_to_1000","1_to_4",3,"male","none",2,"real_estate",35,"stores","rent",1,"skilled",2,"none","yes",1],
["no_checking",37,"prior_payments_delayed","other",6971,"500_to_1000","1_to_4",3,"male","none",3,"savings_insurance",54,"none","own",2,"skilled",1,"yes","yes",1],
["no_checking",14,"all_credits_paid_back","car_new",1525,"500_to_1000","4_to_7",3,"male","none",4,"real_estate",33,"none","own",1,"skilled",1,"none","yes",0],
["less_0",10,"prior_payments_delayed","furniture",4037,"less_100","4_to_7",3,"male","none",3,"savings_insurance",31,"none","rent",1,"skilled",1,"none","yes",1],
["0_to_200",28,"credits_paid_to_date","retraining",1152,"less_100","less_1",2,"female","none",2,"savings_insurance",20,"stores","own",1,"skilled",1,"none","yes",0],
["less_0",17,"credits_paid_to_date","car_new",1880,"less_100","less_1",3,"female","co-applicant",2,"savings_insurance",41,"none","own",1,"skilled",1,"none","yes",0],
["0_to_200",39,"prior_payments_delayed","appliances",5685,"100_to_500","1_to_4",4,"female","none",2,"unknown",37,"none","own",2,"skilled",1,"yes","yes",1],
["no_checking",32,"prior_payments_delayed","radio_tv",5105,"500_to_1000","1_to_4",4,"male","none",5,"savings_insurance",44,"none","own",2,"management_self-employed",1,"none","yes",1],
["no_checking",38,"prior_payments_delayed","appliances",4990,"500_to_1000","greater_7",4,"male","none",4,"car_other",50,"bank","own",2,"unemployed",2,"yes","yes",1],
["less_0",17,"credits_paid_to_date","furniture",1017,"less_100","less_1",2,"female","none",1,"car_other",30,"none","own",1,"skilled",1,"none","yes",0],
["less_0",33,"all_credits_paid_back","car_new",3618,"500_to_1000","4_to_7",2,"male","none",3,"unknown",31,"stores","own",2,"unskilled",1,"none","yes",0],
["less_0",12,"no_credits","car_new",3037,"less_100","less_1",1,"female","none",2,"car_other",31,"stores","own",1,"skilled",1,"none","yes",0],
["no_checking",23,"prior_payments_delayed","furniture",1440,"100_to_500","1_to_4",3,"female","none",3,"real_estate",39,"stores","own",1,"unskilled",1,"yes","yes",0],
["less_0",18,"prior_payments_delayed","retraining",4032,"less_100","1_to_4",2,"female","none",2,"car_other",36,"none","rent",1,"skilled",1,"none","yes",0],
["no_checking",11,"prior_payments_delayed","car_used",944,"greater_1000","1_to_4",3,"male","none",4,"real_estate",35,"none","own",1,"management_self-employed",1,"yes","yes",0],
["no_checking",36,"prior_payments_delayed","appliances",5927,"unknown","greater_7",4,"male","co-applicant",3,"savings_insurance",47,"none","own",2,"skilled",1,"none","yes",1],
["no_checking",50,"outstanding_credit","other",4694,"unknown","greater_7",4,"male","none",4,"unknown",37,"none","own",1,"skilled",2,"yes","yes",1],
["no_checking",32,"prior_payments_delayed","radio_tv",10584,"100_to_500","1_to_4",3,"male","co-applicant",3,"unknown",46,"stores","own",2,"unskilled",2,"yes","yes",0],
["no_checking",41,"prior_payments_delayed","furniture",8900,"500_to_1000","4_to_7",4,"male","co-applicant",3,"car_other",26,"none","free",2,"skilled",1,"yes","yes",1],
["0_to_200",14,"credits_paid_to_date","car_used",1144,"100_to_500","less_1",2,"female","none",2,"real_estate",33,"none","rent",1,"skilled",1,"none","yes",0],
["no_checking",14,"outstanding_credit","appliances",1680,"100_to_500","greater_7",4,"male","none",3,"car_other",47,"none","own",1,"management_self-employed",1,"none","yes",0],
["0_to_200",23,"credits_paid_to_date","retraining",3387,"less_100","less_1",3,"female","none",3,"savings_insurance",28,"none","own",1,"skilled",1,"none","yes",0],
["no_checking",14,"credits_paid_to_date","furniture",1269,"500_to_1000","greater_7",2,"male","none",2,"savings_insurance",39,"none","own",1,"skilled",1,"none","yes",0],
["no_checking",36,"prior_payments_delayed","appliances",9570,"100_to_500","4_to_7",4,"male","co-applicant",3,"car_other",53,"none","free",2,"skilled",1,"yes","yes",0],
["less_0",16,"credits_paid_to_date","car_new",1428,"less_100","4_to_7",1,"male","none",1,"car_other",20,"bank","rent",1,"unemployed",1,"yes","yes",0],
["no_checking",24,"outstanding_credit","car_used",4620,"greater_1000","1_to_4",3,"male","none",4,"savings_insurance",40,"none","own",2,"skilled",1,"yes","yes",0],
["no_checking",34,"prior_payments_delayed","furniture",2196,"500_to_1000","greater_7",3,"male","none",4,"savings_insurance",27,"none","own",1,"skilled",1,"none","yes",0],
["no_checking",25,"prior_payments_delayed","car_used",8708,"100_to_500","1_to_4",4,"male","none",5,"car_other",43,"none","free",2,"management_self-employed",1,"none","yes",0],
["no_checking",37,"outstanding_credit","radio_tv",10550,"unknown","greater_7",5,"male","co-applicant",4,"unknown",48,"stores","own",2,"unemployed",2,"yes","yes",1],
["no_checking",27,"prior_payments_delayed","radio_tv",4981,"500_to_1000","4_to_7",4,"male","none",4,"savings_insurance",47,"none","own",2,"management_self-employed",2,"yes","yes",0],
["less_0",13,"all_credits_paid_back","car_new",2436,"less_100","less_1",2,"female","none",1,"savings_insurance",19,"stores","own",1,"skilled",1,"none","yes",0],
["greater_200",25,"outstanding_credit","appliances",4136,"100_to_500","4_to_7",3,"male","none",2,"car_other",46,"bank","own",1,"unemployed",1,"yes","yes",0],
["no_checking",15,"credits_paid_to_date","retraining",4014,"less_100","1_to_4",4,"male","co-applicant",4,"savings_insurance",33,"none","own",1,"skilled",1,"yes","yes",1],
["no_checking",28,"prior_payments_delayed","appliances",5440,"100_to_500","4_to_7",3,"male","none",2,"unknown",40,"none","own",2,"skilled",1,"yes","yes",1],
["less_0",13,"prior_payments_delayed","appliances",250,"500_to_1000","4_to_7",2,"male","none",3,"car_other",28,"stores","own",1,"skilled",1,"none","yes",0],
["less_0",19,"credits_paid_to_date","furniture",2111,"less_100","4_to_7",3,"male","none",2,"savings_insurance",34,"bank","own",1,"unemployed",2,"none","yes",0],
["no_checking",27,"prior_payments_delayed","appliances",6455,"100_to_500","4_to_7",3,"male","none",4,"car_other",43,"none","own",1,"skilled",1,"none","yes",1],
["less_0",17,"credits_paid_to_date","car_used",250,"less_100","4_to_7",3,"female","none",2,"real_estate",40,"none","free",2,"skilled",1,"none","yes",0],
["no_checking",27,"prior_payments_delayed","radio_tv",4521,"100_to_500","less_1",4,"male","none",4,"savings_insurance",28,"none","own",1,"management_self-employed",2,"yes","yes",0],
["no_checking",37,"prior_payments_delayed","other",7945,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",39,"none","own",2,"management_self-employed",1,"none","yes",0],
["less_0",6,"all_credits_paid_back","car_used",250,"less_100","1_to_4",2,"male","none",2,"savings_insurance",28,"stores","rent",1,"skilled",1,"none","yes",1],
["less_0",14,"all_credits_paid_back","appliances",1431,"less_100","unemployed",1,"female","none",1,"car_other",25,"stores","own",1,"skilled",1,"none","yes",1],
["greater_200",5,"credits_paid_to_date","car_used",250,"less_100","4_to_7",3,"male","none",2,"savings_insurance",42,"none","rent",1,"skilled",1,"none","yes",0]
    ]
)

In [ ]:
subscription.feedback_logging.show_table()

In [ ]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

In [ ]:
subscription.quality_monitoring.get_run_details()

In [ ]:
subscription.quality_monitoring.show_table()

In [ ]:
subscription.quality_monitoring._get_data_from_rest_api()

In [ ]:
ai_client.data_mart.get_deployment_metrics()

## Fairness monitoring

In [ ]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26,120]], minority=[[18,25]], threshold=0.95)
            ],
            prediction_column='prediction',
            favourable_classes=[0],
            unfavourable_classes=[1, 2],
            min_records=10
        )

In [ ]:
subscription.fairness_monitoring.run()

In [ ]:
subscription.fairness_monitoring.show_table()

In [ ]:
subscription.get_details()

## Score the model again now that monitoring is configured

In [ ]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print(scoring_response)

# Create historical data

In [ ]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_1.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_2.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_3.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_4.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_5.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_6.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_7.json

In [ ]:
historyDays = 7

In [ ]:
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

for day in range(historyDays):
    print('Loading day {}'.format(day + 1))
    history_file = 'payload_history_' + str(day + 1) + '.json'
    with open(history_file) as f:
        payloads = json.load(f)
        hourly_records = int(len(payloads) / 24)
        index = 0
        for hour in range(24):
            recordsList = []
            for i in range(hourly_records):
                score_time = str(datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1))))
                recordsList.append(PayloadRecord(request=payloads[index]['request'], response=payloads[index]['response'], scoring_timestamp=score_time))
                index += 1
            subscription.payload_logging.store(records=recordsList)
print('Finished')

In [ ]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

In [ ]:
# store performance monitor history in MeasurementFacts table
import random
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

In [ ]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

## Load historical quality MeasurementFacts to AIOS

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.8,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.8
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

## Load historical fairness data

In [ ]:
!rm fairness_history.json
!rm debiased_fairness_history.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/fairness_history.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/debiased_fairness_history.json

### Fairness data

In [ ]:
# token_data = {
#     'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
#     'response_type': 'cloud_iam',
#     'apikey': AIOS_CREDENTIALS['apikey']
# }

# response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
# iam_token = response.json()['access_token']
# iam_headers = {
#     'Content-Type': 'application/json',
#     'Authorization': 'Bearer %s' % iam_token
# }

# index = 0

# with open('fairness_history.json') as f:
#     payloads = json.load(f)
#     for day in range(historyDays):
#         print('Loading day {}'.format(day + 1))
#         for hour in range(24):
#             score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
#             qualityMetric = {
#                 'metric_type': 'fairness',
#                 'binding_id': binding_uid,
#                 'timestamp': score_time,
#                 'subscription_id': model_uid,
#                 'asset_revision': model_uid,
#                 'deployment_id': deployment_uid,
#                 'value': {
#                     'quality': measurements[day],
#                     'threshold': 0.8,
#                     'metrics': payloads[index]["metrics"]
#                 }
#             }
#             response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
#             index += 1
#         subscription.fairness_monitoring.run()
# print('Finished')

### Debiasied fairness data

In [ ]:
# token_data = {
#     'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
#     'response_type': 'cloud_iam',
#     'apikey': AIOS_CREDENTIALS['apikey']
# }

# response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
# iam_token = response.json()['access_token']
# iam_headers = {
#     'Content-Type': 'application/json',
#     'Authorization': 'Bearer %s' % iam_token
# }

# index = 0

# with open('debiased_fairness_history.json') as f:
#     payloads = json.load(f)
#     for day in range(historyDays):
#         print('Loading day {}'.format(day + 1))
#         for hour in range(24):
#             score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
#             qualityMetric = {
#                 'metric_type': 'debiased_fairness',
#                 'binding_id': binding_uid,
#                 'timestamp': score_time,
#                 'subscription_id': model_uid,
#                 'asset_revision': model_uid,
#                 'deployment_id': deployment_uid,
#                 'value': {
#                     'quality': measurements[day],
#                     'threshold': 0.8,
#                     'metrics': payloads[index]["metrics"]
#                 }
#             }
#             response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
#             index += 1
# print('Finished')

## Run historical fairness monitoring

In [ ]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

metrics_url = 'https://api.aiopenscale.cloud.ibm.com/v1/fairness_monitoring'
request_params = {"fairness_history_run": "true"}
fairness_history_payload = {
    "data_mart_id": data_mart_id,
    "asset_id": model_uid,
    "deployment_id": deployment_uid,
    "fairness_history_run": "true",
    "parameters": {
        "features": [
            {
                "feature": "Sex",
                "majority": ['male'],
                "minority": ['female'],
                "threshold": 0.95
            }, 
            {
                "feature": "Age",
                "majority": [[26,120]],
                "minority": [[0,25]],
                "threshold": 0.95
            }
            ],
        "class_label": "prediction",
        "favourable_class": [0],
        "unfavourable_class": [1,2],
        "min_records": 10
    }
}
response = requests.post(metrics_url, json=fairness_history_payload, headers=iam_headers, params=request_params)
print(response.text)

## Explainability

In [ ]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(
    problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
            input_data_type=InputDataType.STRUCTURED,
            feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            label_column='Risk',
            training_data_reference=BluemixCloudObjectStorageReference(
                COS_CREDENTIALS,
                COS_BUCKET_NAME + '/credit_risk_training.csv',
                first_line_header=True
            )
        )

In [ ]:
subscription.explainability.get_details()

## Explain a transaction

In [ ]:
# subscription.explainability.run('9db3077f-8d4a-4dc4-9c8f-30fde3f3ad8e-1')

In [ ]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)